In [1]:
import pandas as pd
import polars as pl
import numpy as np
import math
import time
from datetime import date
from datetime import datetime

from tqdm import tqdm

from os import listdir
from os.path import isfile, join

import psycopg2

In [2]:
import pymysql
import pandas as pd
from sqlalchemy import create_engine

# def query(sql):
#    host,port,db,user,pw=para()
#    engine=create_engine('mysql+pymysql://{}:{}@{}:{}/{}?charset=utf8'.format(user,pw,host,port,db))
#    return (pd.read_sql_query(sql,engine))

def update(sql):
   host,port,db,user,pw=para()
   con=pymysql.connect(host=host,port=port,db=db,user=user,password=pw,charset='utf8')
   with con.cursor() as cursor:
      print('db_upd:',cursor.execute(sql))
      con.commit()

def update2(sql,value):
   host,port,db,user,pw=para()
   con=pymysql.connect(host=host,port=port,db=db,user=user,password=pw,charset='utf8')
   with con.cursor() as cursor:
      print('db_upd:',cursor.executemany(sql,value))
      con.commit()

def para():
   host='127.0.0.1'
   port=3307
   db='growdb'
   user='growdbro'
   pw='foam374:buts'
   return host,port,db,user,pw

from sqlalchemy import create_engine,text
def query(sql, infer_schema_length=1000):
    host,port,db,user,pw=para()
    engine=create_engine('mysql+pymysql://{}:{}@{}:{}/{}?charset=utf8'.format(user,pw,host,port,db))
    return pl.read_database(sql, connection=engine, infer_schema_length=infer_schema_length)

In [3]:
## 自己評価5段階データ取得
cid = 19558  # GIFT_COURSE ID
sql = f"""
SELECT
	u.ID GROW_ID,
	c.COMPETENCY_TYPE,
	CASE c.COMPETENCY_TYPE
		WHEN 1 THEN '認知'
		WHEN 2 THEN '自己'
		WHEN 3 THEN '他者'
		WHEN 4 THEN 'コミュニティ'
	END COMPETENCY_TYPE_NAME,
	c.ID COMPETENCY_ID,
	c.COMPETENCY_NAME, 
	eq.ID QUESTION_ID, 
	eq.SELF_QUESTION_TEXT QUESTION_TEXT, 
	sefrr.SCORE, 
	sefrr.ANSWER_TIME 
FROM GIFT_COURSES gc 
INNER JOIN USER_GIFT_COURSES ugc ON ugc.GIFT_COURSE_ID = gc.ID AND gc.ID = {cid}
INNER JOIN USERS u ON u.ID = ugc.USER_ID 
INNER JOIN SELF_EVAL_FIVE_RUBRICS_RECORDS sefrr ON sefrr.USER_GIFT_COURSE_ID = ugc.ID
INNER JOIN EVAL_QUESTIONS eq ON eq.ID = sefrr.QUESTION_ID 
INNER JOIN COMPETENCIES c ON eq.BASE_COMPETENCY_ID = c.MASTER_ID
WHERE gc.DELETED_AT IS NULL AND ugc.DELETED_AT IS NULL
ORDER BY u.ID, c.COMPETENCY_TYPE, c.ID, eq.ID;
"""

self_score = query(sql)

In [4]:
self_score.write_csv("./data/自己評価_202601.csv")


In [5]:
sql = f"""
SELECT 
	u.ID EVALUATEE_ID, 
	u2.ID EVALUATOR_ID, 
	c.COMPETENCY_TYPE,
	CASE c.COMPETENCY_TYPE
		WHEN 1 THEN '認知'
		WHEN 2 THEN '自己'
		WHEN 3 THEN '他者'
		WHEN 4 THEN 'コミュニティ'
	END COMPETENCY_TYPE_NAME,
	c.ID COMPETENCY_ID,
	c.COMPETENCY_NAME, 
	eq.ID QUESTION_ID, 
	eq.QUESTION_TEXT, 
	gfrr.SCORE, 
	gfrr.ANSWER_TIME,
	er.NAME RELATION_NAME,
	ugc.ID USER_GIFT_COURSE_ID
FROM GIFT_COURSES gc 
INNER JOIN USER_GIFT_COURSES ugc ON ugc.GIFT_COURSE_ID = gc.ID AND gc.ID = {cid}
INNER JOIN GIFT_FIVE_RUBRICS_RECORDS gfrr ON gfrr.USER_GIFT_COURSE_ID = ugc.ID
INNER JOIN EVAL_QUESTIONS eq ON eq.ID = gfrr.QUESTION_ID 
INNER JOIN COMPETENCIES c ON eq.BASE_COMPETENCY_ID = c.MASTER_ID
INNER JOIN USERS u ON u.ID = gfrr.EVALUATEE_ID
INNER JOIN USERS u2 ON u2.ID = gfrr.EVALUATOR_ID  
INNER JOIN GIFT_REQUESTS gr ON gr.USER_GIFT_COURSE_ID = ugc.ID AND gr.USER_ID = u2.ID
INNER JOIN GUEST_PROFILES gp ON gp.GIFT_REQUEST_ID = gr.ID
INNER JOIN EVALUATOR_RELATIONS er ON er.ID = gp.EVALUATOR_RELATION_ID 
WHERE gc.DELETED_AT IS NULL AND ugc.DELETED_AT IS NULL
ORDER BY u.ID, u2.ID, c.COMPETENCY_TYPE, c.ID, eq.ID;
"""

gift_score = query(sql)

In [6]:
gift_score

EVALUATEE_ID,EVALUATOR_ID,COMPETENCY_TYPE,COMPETENCY_TYPE_NAME,COMPETENCY_ID,COMPETENCY_NAME,QUESTION_ID,QUESTION_TEXT,SCORE,ANSWER_TIME,RELATION_NAME,USER_GIFT_COURSE_ID
i64,i64,i64,str,i64,str,i64,str,i64,i64,str,i64
1572037,1572137,1,"""認知""",1,"""課題設定""",991,"""彼・彼女は、問題の原因究明のために必要な情報を十分に収集・活…",4,2225,"""部下""",1821142
1572037,1572137,1,"""認知""",1,"""課題設定""",992,"""彼・彼女は、他人が見過ごしがちな問題を指摘している""",4,2759,"""部下""",1821142
1572037,1572137,1,"""認知""",1,"""課題設定""",993,"""彼・彼女は、多面的に分析することで問題の原因を突き止めている""",5,6393,"""部下""",1821142
1572037,1572137,1,"""認知""",2,"""解決意向""",994,"""彼・彼女は、課題に適した具体的な計画を立てている""",5,2291,"""部下""",1821142
1572037,1572137,1,"""認知""",2,"""解決意向""",995,"""彼・彼女は、進行中の計画でも改善の余地を十分模索している""",4,1383,"""部下""",1821142
…,…,…,…,…,…,…,…,…,…,…,…
1572136,1572338,4,"""コミュニティ""",24,"""組織へのコミットメント""",1061,"""彼・彼女は、組織に対して献身的な姿勢を示している""",4,27783,"""上司""",1821241
1572136,1572338,4,"""コミュニティ""",24,"""組織へのコミットメント""",1062,"""彼・彼女は、組織の利益と繁栄を第一に考え行動している""",4,42927,"""上司""",1821241
1572136,1572338,4,"""コミュニティ""",25,"""誠実さ""",1063,"""彼・彼女は、ルールを守らない人に対して守るよう働きかけている""",4,16915,"""上司""",1821241


In [7]:
gift_score.write_csv("アイン_素点_202601.csv")

In [11]:
from pathlib import Path

output_path = Path("./data")
output_path.mkdir(parents=True, exist_ok=True)

gift_score.write_csv(output_path / "アイン_素点_202601.csv")

In [8]:
pwd

'/Users/c.yokota'